## MLflow's Model Registry

In [5]:
import os
import mlflow
from mlflow.tracking import MlflowClient

In [7]:
os.environ['MLFLOW_TRACKING_PASSWORD'] = '***REMOVED***'
os.environ['MLFLOW_TRACKING_URI']="***REMOVED***"
os.environ['MLFLOW_TRACKING_USERNAME']="***REMOVED***"

In [32]:
EXPT_NAME = 'final-goal'
MLFLOW_TRACKING_URI=os.environ['MLFLOW_TRACKING_URI']
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPT_NAME)

In [9]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [33]:
from mlflow.entities import ViewType
experiment = client.get_experiment_by_name(EXPT_NAME)

runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    filter_string="metrics.accuracy > 0.9",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=2,
    order_by=["metrics.accuracy DESC"]
)
for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.***REMOVED***.metrics['accuracy']:.4f}")

best_run, second_best = runs[0], runs[1]

run id: d92d79a7278d4408b2406d82b2b09aab, accuracy: 0.9923
run id: dccf3c615e1348e7b8c4b4fd5af71339, accuracy: 0.9827


In [34]:
i = 0
for run in runs:
    i = i+1
    mlflow.register_model( 
            model_uri = f"runs:/{run.info.run_id}/model",
            name = f'fake-news-detect-{i}'
    )

Successfully registered model 'fake-news-detect-1'.
2022/09/01 10:06:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: fake-news-detect-1, version 1
Created version '1' of model 'fake-news-detect-1'.
Successfully registered model 'fake-news-detect-2'.
2022/09/01 10:06:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: fake-news-detect-2, version 1
Created version '1' of model 'fake-news-detect-2'.


In [26]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 4, stage: None


In [32]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [33]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test ***REMOVED***set, which corresponds to the NYC Green Taxi ***REMOVED*** from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training ***REMOVED*** and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [34]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_***REMOVED***frame(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [36]:
df = read_***REMOVED***frame("***REMOVED***/green_trip***REMOVED***_2021-03.csv")

/var/folders/42/f9s_rgk15078ym2w50_xtc180000gq/T/ipykernel_5486/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [41]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)